# Week 1 - Measuring Meaning & Sampling
This week, we begin by "begging, borrowing and stealing" text from several
contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for
machines to "read" and analyze so that we can begin to build our sample. This notebook outlines scraping text from the web, PDF and Word documents. Then we detail "spidering" or walking
through hyperlinks to build samples of online content, and using APIs,
Application Programming Interfaces, provided by webservices to access their
content. Along the way, we will use regular expressions, outlined in the
reading, to remove unwanted formatting and ornamentation. Next, we discuss
various text encodings, filtering and data structures in which text can be
placed for analysis. Finally, we ask you to begin building a corpus for preliminary analysis and articulate what your sample represents in context of your final project.

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# for setting up github key
# git config --add --local core.sshCommand 'ssh -i <<<PATH_TO_SSH_KEY>>>'
!git config --global user.email "beilrz@uchicago.edu"
!git config --global user.name "beilrz"

Mounted at /content/drive


We made a python package just for this course: lucem_illud. If you haven't installed this package, you should run the following code first. You don't need to install the package later; all you need to do is just to import the package with: import lucem_illud. For your final projects, you may find it useful to [read the lucem_illud source code](https://github.com/UChicago-Computational-Content-Analysis/lucem_illud/tree/main/lucem_illud) and modify your code for your own interests.

In [2]:
!pip install git+https://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git
#installing lucem_illud package
#lucem_illud is a Latin phrase meaning "that light", the insight we can discover in text data!
#If you get an error like "Access is denied", try running the `pip` command on the command line as an administrator.

  Cloning https://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git to /tmp/pip-req-build-_hcl4hvw
  Running command git clone --filter=blob:none --quiet https://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git /tmp/pip-req-build-_hcl4hvw
  Resolved https://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git to commit 9fa6f1abdc19d159d587b4017ee6897fe7a18d4d
  Preparing metadata (setup.py) ... done


If you're not familiar with jupyter notebook, you may wonder what the exclamation mark(!) at the beginning of the command does (or even what pip means). The exclamation mark enables us to execute Terminal commands in the notebook cells (e.g., run `!ls` to display files in the current folder).

There is also a special download required by the `lucem_illud` module in the module `spacy`. You will see this 'en' module later, but you should probably run the following 2 lines of code so you can import `lucem_illud`.

In [3]:
import spacy
!python -m spacy download en

2024-01-12 00:32:49.664473: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-12 00:32:49.664565: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-12 00:32:49.666984: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-12 00:32:51.696950: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 28.6 MB/s eta 0:00:00
✔ Download and installation successf

For this notebook we will be using the following packages:

In [4]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud #pip install git+https://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git

#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx #reading MS doc files, install as `python-docx`

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python
import re #for regexs
import urllib.parse #For joining urls
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories

We will also be working on the following files/urls

In [5]:
wikipedia_base_url = 'https://en.wikipedia.org'
wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
content_analysis_save = 'wikipedia_content_analysis.html'
example_text_file = 'sometextfile.txt'
information_extraction_pdf = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/Content%20Analysis%2018.pdf'
example_docx = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/macs6000_connecting_to_midway.docx'
example_docx_save = 'example.docx'

# Scraping

Before we can start analyzing content we need to obtain it. Sometimes it will be
provided to us from a pre-curated text archive, but sometimes we will need to
download it. As a starting example we will attempt to download the wikipedia
page on content analysis. The page is located at [https://en.wikipedia.org/wiki/
Content_analysis](https://en.wikipedia.org/wiki/Content_analysis) so lets start
with that.

We can do this by making an HTTP GET request to that url, a GET request is
simply a request to the server to provide the contents given by some url. The
other request we will be using in this class is called a POST request and
requests the server to take some content we provide. While the Python standard
library does have the ability do make GET requests we will be using the
[_requests_](http://docs.python-requests.org/en/master/) package as it is _'the
only Non-GMO HTTP library for Python'_...also it provides a nicer interface.

In [6]:
#wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
requests.get(wikipedia_content_analysis)

<Response [200]>

`'Response [200]'` means the server responded with what we asked for. If you get
another number (e.g. 404) it likely means there was some kind of error, these
codes are called HTTP response codes and a list of them can be found
[here](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes). The response
object contains all the data the server sent including the website's contents
and the HTTP header. We are interested in the contents which we can access with
the `.text` attribute.

In [7]:
wikiContentRequest = requests.get(wikipedia_content_analysis)
print(wikiContentRequest.text[:1000])

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-client-prefs-pinned-disabled vector-toc-available" lang="en" dir="ltr">
<head>
<meta charset="UTF-8">
<title>Content analysis - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-cli

This is not what we were looking for, because it is the start of the HTML that
makes up the website. This is HTML and is meant to be read by computers. Luckily
we have a computer to parse it for us. To do the parsing we will use [_Beautiful
Soup_](https://www.crummy.com/software/BeautifulSoup/) which is a better parser
than the one in the standard library.

But before we proceed to Beautiful Soup, a digression about Python syntax, especially about objects and functions.
For those who are not familiar with the syntax of python (or, if you're familiar with R programming), you might wonder what requests.get or wikiContentRequest.text mean. To understand this, you need to first understand what objects are. You may have heard that Python is an object oriented programming language (unlike the procedure oriented programming language, an example of which is R). Object is a set of variables (or, data) and functions into which you pass your data. So, in object oriented programming languages, like python, variables and functions are bunleded into objects.

For example, let's look at wikiContentRequest. We use dir() function, which returns the list of attributes and functions of objects.

In [8]:
 dir(wikiContentRequest)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

There's 'text' here. We used 'wikiContentRequest.text' to access 'text.' In other words, we use .(dot notation) to access functions from objects. wikiContentRequest has a set of functions, as shown above, and we used 'wikiContentRequest.text' to access one of them. By the way, dot notations do not necessarily refer to functions--it refers to anything that the entity contains.



Moving on to the next step: BeautifulSoup, a Python library which extracts data from HTML and XML, and transforms HTML files into Python objects.

In [9]:
wikiContentSoup = bs4.BeautifulSoup(wikiContentRequest.text, 'html.parser')
print(wikiContentSoup.text[:200])





Content analysis - Wikipedia




































Jump to content







Main menu





Main menu
move to sidebar
hide



		Navigation
	


Main pageContentsCurrent eventsRandom arti


This is better but there's still random whitespace and we have more than just
the text of the article. This is because what we requested is the whole webpage,
not just the text for the article.

We want to extract only the text we care about, and in order to do this we will
need to inspect the html. One way to do this is simply to go to the website with
a browser and use its inspection or view source tool. If javascript or other
dynamic loading occurs on the page, however, it is likely that what Python
receives is not what you will see, so we will need to inspect what Python
receives. To do this we can save the html `requests` obtained.

In [10]:
#content_analysis_save = 'wikipedia_content_analysis.html'

with open(content_analysis_save, mode='w', encoding='utf-8') as f:
    f.write(wikiContentRequest.text)

open() is a function which literally opens and returns the file. This function has multiple modes, and, here, we used mode as 'w', which means: open a file for writing. And then, we use 'write' function to write on the empty file (content_analysis_save) that we created using open(content_analysis_save, mode='w', encoding='utf-8').} What did we write on this file? The text we got from wikiContentRequest.text

Now let's open the file (`wikipedia_content_analysis.html`) we just created with
a web browser. It should look sort of like the original but without the images
and formatting.

As there is very little standardization on structuring webpages, figuring out
how best to extract what you want is an art. Looking at this page it looks like
all the main textual content is inside `<p>`(paragraph) tags within the `<body>`
tag.

In [11]:
contentPTags = wikiContentSoup.body.findAll('p')
for pTag in contentPTags[:3]:
    print(pTag.text)

Content analysis is the study of documents and communication artifacts, which might be texts of various formats, pictures, audio or video. Social scientists use content analysis to examine patterns in communication in a replicable and systematic manner.[1] One of the key advantages of using content analysis to analyse social phenomena is their non-invasive nature, in contrast to simulating social experiences or collecting survey answers.

Practices and philosophies of content analysis vary between academic disciplines. They all involve systematic reading or observation of texts or artifacts which are assigned labels (sometimes called codes) to indicate the presence of interesting, meaningful pieces of content.[2][3] By systematically labeling the content of a set of texts, researchers can analyse patterns of content quantitatively using statistical methods, or use qualitative methods to analyse meanings of content within texts.

Computers are increasingly used in content analysis to au

Another excursion for those who are not familiar with programming: for loop. For loop is used to iterate over a sequence. "ContentPTags" contains multiple paragraphs, each of which starts and ends with `<p>`. What the "for pTag in contentPtags[:3]" does here is: find each paragraph in contentPTags, which, here, we limited to the first three using contentPtags[:3], and then print each paragraph. So, we have three paragraphs. By the way, you can insert `<p>` in juputer notebook!

We now have all the text from the page, split up by paragraph. If we wanted to
get the section headers or references as well it would require a bit more work,
but is doable.

There is one more thing we might want to do before sending this text to be
processed, remove the references indicators (`[2]`, `[3]` , etc). To do this we
can use a short regular expression (regex).

In [12]:
contentParagraphs = []
for pTag in contentPTags:
    #strings starting with r are raw so their \'s are not modifier characters
    #If we didn't start with r the string would be: '\\[\\d+\\]'
    contentParagraphs.append(re.sub(r'\[\d+\]', '', pTag.text))

#convert to a DataFrame
contentParagraphsDF = pandas.DataFrame({'paragraph-text' : contentParagraphs})
print(contentParagraphsDF)

                                       paragraph-text
0   Content analysis is the study of documents and...
1   Practices and philosophies of content analysis...
2   Computers are increasingly used in content ana...
3   Content analysis is best understood as a broad...
4   The simplest and most objective form of conten...
5   A further step in analysis is the distinction ...
6   Quantitative content analysis highlights frequ...
7   Siegfried Kracauer provides a critique of quan...
8   The data collection instrument used in content...
9   According to current standards of good scienti...
10  Furthermore, the Database of Variables for Con...
11  With the rise of common computing facilities l...
12  By having contents of communication available ...
13  Computer-assisted analysis can help with large...
14  Robert Weber notes: "To make valid inferences ...
15  According to today's reporting standards, quan...
16  There are five types of texts in content analy...
17  Content analysis is rese

Since we learned how to do for loop, you might get what we did here: using contentParagraphs = [], we made an empty list; and then, for each paragraph in contentPTags, we substituted every [\d+\] with '', i.e., removed every [\d+\], and then appended each paragraph (now without [\d+\]) to the empty list. As we can see, we have a dataframe, each row of which is each paragraph of contentPTags, without reference indicators.

By the way, what does [\d+\] mean? If you are not familiar with regex, it is a way of specifying searches in text.
A regex engine takes in the search pattern, in the above case `'\[\d+\]'` and
some string, the paragraph texts. Then it reads the input string one character
at a time checking if it matches the search. Here the regex `'\d'` matches
number characters (while `'\['` and `'\]'` capture the braces on either side).

Now we have a `DataFrame` containing all relevant text from the page ready to be processed

In [13]:
findNumber = r'\d'
regexResults = re.search(findNumber, 'not a number, not a number, numbers 2134567890, not a number')
regexResults

<re.Match object; span=(36, 37), match='2'>

In Python the regex package (`re`) usually returns `Match` objects (you can have
multiple pattern hits in a a single `Match`), to get the string that matched our
pattern we can use the `.group()` method, and as we want the first one we will
ask for the 0'th group.

In [14]:
print(regexResults.group(0))

2


That gives us the first number, if we wanted the whole block of numbers we can
add a wildcard `'+'` which requests 1 or more instances of the preceding
character.

In [15]:
findNumbers = r'\d+'
regexResults = re.search(findNumbers, 'not a number, not a number, numbers 2134567890, not a number')
print(regexResults.group(0))

2134567890


Now we have the whole block of numbers, there are a huge number of special
characters in regex, for the full description of Python's implementation look at
the [re docs](https://docs.python.org/3/library/re.html) there is also a short
[tutorial](https://docs.python.org/3/howto/regex.html#regex-howto).

# <font color=" #93FFE8">Exercise 1</font>
<font color="#93FFE8">Construct cells immediately below this that describe and download webcontent relating to your anticipated final project. Use beautiful soup and at least five regular expressions to extract relevant, nontrivial *chunks* of that content (e.g., cleaned sentences, paragraphs, etc.) to a pandas `Dataframe`.</font>

For my final project, I propose to develop a type of spider that crawl the internet for related content by modeling a trust or worldview parameter. For example, if I start from a pro-Trump article, I could find links to other political figures, such as Biden, Republican and other Trump’s allies. For these links, I will examine them through the model of trust/worldview, and I should find that the article discussing Biden has a different worldview than the article discussing Trump or Republican figures (assuming they are all neutral articles), and less likly to be trusted by the audience of Trump article. Once a source is deemed as trustworthy, it could serve a new start point to gather other sources iteratively. Therefore, I would be able to extract a range of articles that share the similar worldview or pertain to a similar interest, and these articles or pages can be used for further analysis. After collecting more article, the initial model regarding the worldview/trust can be updated accordingly.

This is a just an initial idea. The high-level intuition is that people or content share the similar worldview tend to cluster together, and, by using a tool to evaluate whether a source conform to a specific view, we can efficiently gather sources or users share the same view, and these sources/users can be used for further goals, such as monitoring the situation or perform content analysis or network analysis.



In [16]:
# in this page, I will start from the wikipedia page of QAnon, in order to observe
# if extract content is related to the original opinion likly hold by QAnon people
# https://en.wikipedia.org/wiki/QAnon

import requests
import bs4

# Make an HTTP request
url = 'https://en.wikipedia.org/wiki/QAnon'
response = requests.get(url)

# Parse the HTML
soup = bs4.BeautifulSoup(response.text, 'html.parser')

In [17]:
# extract citaiton source name
def extract_citation_source(soup):
  contentTags = soup.body.findAll("li")
  content = []

  # match between quotation marks
  re_pattern = re.compile(r'"([^"]*)"')
  for tag in contentTags:
    matches = re_pattern.findall(tag.text)

    if len(matches) > 0:
      content.append(matches[0])

  return content

urls_in_html = extract_citation_source(soup)
urls_in_html

['anons',
 'anons',
 'the Storm',
 'the Storm',
 'smoking gun',
 'bombshell',
 'never be forgotten',
 "won't be around much longer",
 'next',
 'terrible May',
 'domino effect',
 'Baked Alaska',
 "Epstein didn't kill himself",
 'Q',
 "HRC extradition already in motion effective yesterday with several countries in case of cross border run. Passport approved to be flagged effective 10/30 @ 12:01\xa0am. Expect massive riots organized in defiance and others fleeing the US to occur. US M's will conduct the operation while NG activated. Proof check: Locate an NG member and ask if activated for duty 10/30 across most major cities.",
 'Original Q',
 'Follow the White rabbit',
 'shadow President',
 'The Storm Is the New Pizzagate – Only Worse',
 'The next-generation bots interfering with the US election',
 'The next-generation bots interfering with the US election',
 'The far right is struggling to contain Qanon after giving it life',
 'Pence shares picture of himself meeting a SWAT officer with

In [18]:
# extract four digital numbers (mostly year)
def extract_four_digits_number(soup):
  contentTags = soup.body.findAll("p")
  content = []

  # match four digits number
  re_pattern = re.compile(r'\b\d{4}\b')
  for tag in contentTags:
    matches = re_pattern.findall(tag.text)

    if len(matches) > 0:
      content.append(matches[0])

  return content

four_digits_in_html = extract_four_digits_number(soup)
four_digits_in_html

['2017',
 '2016',
 '2017',
 '2020',
 '2016',
 '2016',
 '2017',
 '2017',
 '2017',
 '2017',
 '2018',
 '2020',
 '2017',
 '2020',
 '2020',
 '2020',
 '2020',
 '2020',
 '2018',
 '2020',
 '2020',
 '2018',
 '2020',
 '2022',
 '2021',
 '2020',
 '2020',
 '2018',
 '2022',
 '2018',
 '2022',
 '2018',
 '2022',
 '2022',
 '2022',
 '2021',
 '2020',
 '2019',
 '2021',
 '2020',
 '2021',
 '2022',
 '2018',
 '1964',
 '2018',
 '2022',
 '1903',
 '2021',
 '2020',
 '2020',
 '2018',
 '2018',
 '2020',
 '2020',
 '2021',
 '2021',
 '2021',
 '2020',
 '2020',
 '1871',
 '2021',
 '2021',
 '2022',
 '2018',
 '2017',
 '2018',
 '2018',
 '2019',
 '2018',
 '2020',
 '2020',
 '2020',
 '2019',
 '2020',
 '2020',
 '2020',
 '2020',
 '2020',
 '2020',
 '2020',
 '2022',
 '2020',
 '2016',
 '2019',
 '2020',
 '2020',
 '2021',
 '2021',
 '2021',
 '2020',
 '2021',
 '2018',
 '2020',
 '2022',
 '2018',
 '2020',
 '2018',
 '2020',
 '2020',
 '2020',
 '2021',
 '2021',
 '2021',
 '2022']

In [19]:
# extract hashtaged words
def extract_hashtags(soup):
  contentTags = soup.body.findAll("p")
  content = []

  # match four digits number
  re_pattern = re.compile(r'#\w+')
  for tag in contentTags:
    matches = re_pattern.findall(tag.text)

    if len(matches) > 0:
      content.append(matches[0])

  return content

hashtaged_words = extract_hashtags(soup)
hashtaged_words

['#SaveTheChildren',
 '#SaveTheChildren',
 '#voterfraud',
 '#TakeTheOath',
 '#WWG1WGA',
 '#WWG1WGA']

In [20]:
# extract quoted text in main body, and remove references indicators
def extract_quoted_text(soup):
  contentTags = soup.body.findAll("p")
  content = []

  # match four digits number
  re_pattern = re.compile(r'"([^"]*)"')
  for tag in contentTags:
    matches = re_pattern.findall(tag.text)

    if len(matches) > 0:
      content.append(re.sub(r'\[\d+\]', "", matches[0]))

  return content

quoted_text = extract_quoted_text(soup)
quoted_text

['Q',
 'the Storm',
 'drops',
 'while Q has a number of precursor conspiracy theories and scams\xa0... no conspiracy theory feeds more immediately into Q than Pizzagate',
 'Human Trafficking General',
 'Storm',
 'anon',
 'Q Clearance Patriot',
 'detained',
 'clues',
 'Frazzledrip',
 'BaruchtheScribe',
 'Pizzagate on steroids',
 'drops',
 'primary foreign actor touting QAnon-narratives online',
 'strong and growing',
 'Soviet Citizens',
 'one of the most popular QAnon Twitter accounts in the world',
 'arrest',
 'QAnon Costa Rica',
 'drops',
 'deep state',
 'clues',
 'disinformation is necessary',
 'drop',
 'patriots',
 'the Storm',
 'adrenochrome harvesting',
 'big tent conspiracy theory',
 'vaccines [not all]',
 'secret United States biolabs',
 'The dynamics of the invasion are shifting their views. In an astoundingly short space of time, Xi Jinping appears to have been recast from a villain to a hero in the QAnon conspiracy pantheon.',
 'lizard people',
 'drops',
 'distinct signatures

In [21]:
# extract all lines the word "Trump"
def extract_Trump(soup):
  contentTags = soup.body.findAll("p")
  content = []

  # match four digits number
  re_pattern = re.compile(r'\bTrump\b', re.IGNORECASE)
  for tag in contentTags:
    sentences = tag.text.split(".")

    for s in sentences:
      if re_pattern.search(s):
        cleaned_sentence = re.sub(r'\[\d+\]', "", s)
        cleaned_sentence = cleaned_sentence.strip()
        content.append(cleaned_sentence)


  return content

trump_sentence = extract_Trump(soup)
trump_sentence

['Their core belief is that a cabal of Satanic, cannibalistic child molesters are operating a global child sex trafficking ring which conspired against Donald Trump',
 'Followers believe the Trump administration secretly fought the cabal of pedophiles, and would conduct arrests and executions of thousands of cabal members on a day known as "the Storm" or "the Event"',
 "QAnon has also claimed that Trump stimulated the conspiracy of Russian interference in the 2016 United States presidential election to enlist Robert Mueller to join him in exposing the sex trafficking ring, and to prevent a coup d'état by Barack Obama, Hillary Clinton, and George Soros",
 'Q claimed to be a high-level government official with Q clearance, with access to classified information about the Trump administration and its opponents',
 'QAnon followers began to appear at Trump campaign rallies in August 2018, and Trump amplified QAnon accounts on Twitter',
 "Members of the movement took part in the 2020 United S

In [22]:
#convert to a DataFrame
text_data = [urls_in_html, hashtaged_words, four_digits_in_html, quoted_text, trump_sentence]
text_labels = ['citation_source', 'hashtaged_words', 'years_text', 'quoted_text', 'Trump_text']

# Flatten the list of lists while associating each element with its label
flattened_data = [(elem, label) for sublist, label in zip(text_data, text_labels) for elem in sublist]

# Create a DataFrame
df = pandas.DataFrame(flattened_data, columns=['Text', 'Label'])
df

,Text,Label
0,anons,citation_source
1,anons,citation_source
2,the Storm,citation_source
3,the Storm,citation_source
4,smoking gun,citation_source
...,...,...
837,"In May 2021, Powell asserted that Trump ""can s...",Trump_text
838,Former Arizona State Senator Kelly Townsend is...,Trump_text
839,"In 2020, she worked with Corsi again, claiming...",Trump_text
840,"In the lead-up to January 6, 2021, Townsend sp...",Trump_text



# Spidering

What if we want to to get a bunch of different pages from wikipedia. We would
need to get the url for each of the pages we want. Typically, we want pages that
are linked to by other pages and so we will need to parse pages and identify the
links. Right now we will be retrieving all links in the body of the content
analysis page.

To do this we will need to find all the `<a>` (anchor) tags with `href`s
(hyperlink references) inside of `<p>` tags. `href` can have many
[different](http://stackoverflow.com/questions/4855168/what-is-href-and-why-is-
it-used) [forms](https://en.wikipedia.org/wiki/Hyperlink#Hyperlinks_in_HTML) so
dealing with them can be tricky, but generally, you will want to extract
absolute or relative links. An absolute link is one you can follow without
modification, while a relative link requires a base url that you will then
append. Wikipedia uses relative urls for its internal links: below is an example
for dealing with them.

In [23]:
#wikipedia_base_url = 'https://en.wikipedia.org'

otherPAgeURLS = []
#We also want to know where the links come from so we also will get:
#the paragraph number
#the word the link is in
for paragraphNum, pTag in enumerate(contentPTags):
    #we only want hrefs that link to wiki pages
    tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
    for aTag in tagLinks:
        #We need to extract the url from the <a> tag
        relurl = aTag.get('href')
        linkText = aTag.text
        #wikipedia_base_url is the base we can use the urllib joining function to merge them
        #Giving a nice structured tupe like this means we can use tuple expansion later
        otherPAgeURLS.append((
            urllib.parse.urljoin(wikipedia_base_url, relurl),
            paragraphNum,
            linkText,
        ))
print(otherPAgeURLS[:10])

[('https://en.wikipedia.org/wiki/Document', 0, 'documents'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Coding_(social_sciences)', 1, 'assigned labels (sometimes called codes)'), ('https://en.wikipedia.org/wiki/Semantics', 1, 'meaningful'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Quantitative_research', 1, 'quantitatively'), ('https://en.wikipedia.org/wiki/Statistics', 1, 'statistical methods'), ('https://en.wikipedia.org/wiki/Qualitative_research', 1, 'qualitative'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Machine_learning', 2, 'Machine learning')]


In [24]:
print(contentPTags)

[<p><b>Content analysis</b> is the study of <a href="/wiki/Document" title="Document">documents</a> and communication artifacts, which might be texts of various formats, pictures, audio or video. Social scientists use content analysis to examine patterns in communication in a replicable and systematic manner.<sup class="reference" id="cite_ref-1"><a href="#cite_note-1">[1]</a></sup> One of the key advantages of using content analysis to analyse social phenomena is their non-invasive nature, in contrast to simulating social experiences or collecting survey answers.
</p>, <p>Practices and philosophies of content analysis vary between academic disciplines. They all involve systematic reading or observation of <a href="/wiki/Text_(literary_theory)" title="Text (literary theory)">texts</a> or artifacts which are <a href="/wiki/Coding_(social_sciences)" title="Coding (social sciences)">assigned labels (sometimes called codes)</a> to indicate the presence of interesting, <a href="/wiki/Semant

Another excursion: Why do we use enumerate() here? enumerate() takes a collection, enumerates, and returns an enumate object with both the numbers and the collection. For example, contentPTags (the collection we used here) is comprised of paragraphs. We want the paragraph number of each paragraph. And this is what enumerate() does: it returns the paragraph number and the paragraph.

We will be adding these new texts to our DataFrame `contentParagraphsDF` so we
will need to add 2 more columns to keep track of paragraph numbers and sources.

In [25]:
contentParagraphsDF['source'] = [wikipedia_content_analysis] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['paragraph-number'] = range(len(contentParagraphsDF['paragraph-text']))

contentParagraphsDF

,paragraph-text,source,paragraph-number
0,Content analysis is the study of documents and...,https://en.wikipedia.org/wiki/Content_analysis,0
1,Practices and philosophies of content analysis...,https://en.wikipedia.org/wiki/Content_analysis,1
2,Computers are increasingly used in content ana...,https://en.wikipedia.org/wiki/Content_analysis,2
3,Content analysis is best understood as a broad...,https://en.wikipedia.org/wiki/Content_analysis,3
4,The simplest and most objective form of conten...,https://en.wikipedia.org/wiki/Content_analysis,4
5,A further step in analysis is the distinction ...,https://en.wikipedia.org/wiki/Content_analysis,5
6,Quantitative content analysis highlights frequ...,https://en.wikipedia.org/wiki/Content_analysis,6
7,Siegfried Kracauer provides a critique of quan...,https://en.wikipedia.org/wiki/Content_analysis,7
8,The data collection instrument used in content...,https://en.wikipedia.org/wiki/Content_analysis,8
9,According to current standards of good scienti...,https://en.wikipedia.org/wiki/Content_analysis,9


Then we can add two more columns to our `Dataframe` and define a function to
parse
each linked page and add its text to our DataFrame.

In [26]:
contentParagraphsDF['source-paragraph-number'] = [None] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['source-paragraph-text'] = [None] * len(contentParagraphsDF['paragraph-text'])

def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    #Make a dict to store data before adding it to the DataFrame
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        parsDict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(parsDict)

And run it on our list of link tags

In [27]:
for urlTuple in otherPAgeURLS[:3]:
    #ignore_index means the indices will not be reset after each append
    contentParagraphsDF = contentParagraphsDF.append(getTextFromWikiPage(*urlTuple),ignore_index=True)
contentParagraphsDF

,paragraph-text,source,paragraph-number,source-paragraph-number,source-paragraph-text
0,Content analysis is the study of documents and...,https://en.wikipedia.org/wiki/Content_analysis,0,None,None
1,Practices and philosophies of content analysis...,https://en.wikipedia.org/wiki/Content_analysis,1,None,None
2,Computers are increasingly used in content ana...,https://en.wikipedia.org/wiki/Content_analysis,2,None,None
3,Content analysis is best understood as a broad...,https://en.wikipedia.org/wiki/Content_analysis,3,None,None
4,The simplest and most objective form of conten...,https://en.wikipedia.org/wiki/Content_analysis,4,None,None
...,...,...,...,...,...
58,Much of qualitative coding can be attributed t...,https://en.wikipedia.org/wiki/Coding_(social_s...,8,1,assigned labels (sometimes called codes)
59,Coding is considered a process of discovery an...,https://en.wikipedia.org/wiki/Coding_(social_s...,9,1,assigned labels (sometimes called codes)
60,"The process can be done manually, which can be...",https://en.wikipedia.org/wiki/Coding_(social_s...,10,1,assigned labels (sometimes called codes)
61,After assembling codes it is time to organize ...,https://en.wikipedia.org/wiki/Coding_(social_s...,11,1,assigned labels (sometimes called codes)



# <font color="93FFE8">Exercise 2</font>
<font color="93FFE8">Construct cells immediately below this that spider webcontent from another site with content relating to your anticipated final project. Specifically, identify urls on a core page, then follow and extract content from them into a pandas `Dataframe`. In addition, demonstrate a *recursive* spider, which follows more than one level of links (i.e., follows links from a site, then follows links on followed sites to new sites, etc.), making sure to define a reasonable endpoint so that you do not wander the web forever :-).</font>



For this part, I crawl the news related to Donald Trump on apnews.com. Start from a hub page, I demostrate a recursive spider with depth 2 to collect 10 pages within the domain of apnews.com/article.

In [ ]:
# function to process a single page
def process_page(url):
  # request page
  r = requests.get(url)
  soup = bs4.BeautifulSoup(r.text, 'html.parser')

  # process page for url
  urls = []
  for x in soup.findAll("a", href=re.compile('/apnews.com/article')):
    urls.append(x.get("href"))

  # process page for content
  content = []
  for x in soup.findAll(["p", "h1", "h2", "h3", "h4", "h5"]):
    if not re.fullmatch(r'^\s*$', x.text) and "Copyright" not in x.text:
      content.append(x.text.strip())

  return (urls, content, soup.title.string)

In [29]:
from collections import deque

# recursive spider function, bfs
def recursive_spider(start_point, page_limit, depth):
  url_stack = deque()
  url_stack.append((start_point, 0)) # (url, depth)

  page_content = [] # (page_title, content_list)
  processed_url = set()
  while len(url_stack) > 0 and len(processed_url) < page_limit:
    curUrl, curDepth = url_stack.pop()
    newUrls, newContent, title = process_page(curUrl)
    processed_url.add(curUrl) # to prevent loop

    # add new content
    page_content += [(x, title) for x in newContent]

    # add new url
    if curDepth + 1 < depth:
      for newUrl in newUrls:
        if newUrl not in processed_url:
          url_stack.append((newUrl, curDepth + 1))

  return page_content

start_point = "https://apnews.com/hub/donald-trump"
page_content = recursive_spider(start_point, 10, 2)

In [30]:
df_spider = pandas.DataFrame(page_content, columns=["content", "title"])
df_spider

,content,title
0,Donald Trump,Donald Trump | Breaking News & Latest Updates ...
1,Fox town hall with Trump draws more TV viewers...,Donald Trump | Breaking News & Latest Updates ...
2,Former Maryland Gov. Larry Hogan steps down fr...,Donald Trump | Breaking News & Latest Updates ...
3,Trump skips much of the final week before Iowa...,Donald Trump | Breaking News & Latest Updates ...
4,Georgia Senate nominates former senator as fif...,Donald Trump | Breaking News & Latest Updates ...
...,...,...
687,“Mr. Epps was one of many who trespassed outsi...,"Ray Epps, a target of Jan. 6 conspiracy theori..."
688,"More than 1,200 defendants have been charged w...","Ray Epps, a target of Jan. 6 conspiracy theori..."
689,Epps once served as an Arizona chapter leader ...,"Ray Epps, a target of Jan. 6 conspiracy theori..."
690,Oath Keepers founder Stewart Rhodes and other ...,"Ray Epps, a target of Jan. 6 conspiracy theori..."


In [31]:
# print all page titles
df_spider["title"].unique()

array(['Donald Trump | Breaking News & Latest Updates | AP News',
       'Defamation trial: Judge blocks Trump lawyers from arguing about E. Jean Carroll’s rape claim | AP News',
       "Iowa's Christian conservatives follow their faith when voting, and some say it leads them to Trump | AP News",
       'Trump is raising expectations heading into the Iowa caucuses | AP News',
       'Biden condemns white Supremacy at site of church shooting in South Carolina | AP News',
       "Judge in Trump's DC election case is latest high-profile victim of swatting  | AP News",
       'Trump asks Maine judge for pause to let Supreme Court rule on ballot access | AP News',
       "In court, Trump's immunity claim raised questions | AP News",
       'How the animated character Marlon could help Trump win Iowa’s caucuses | AP News',
       'Ray Epps, a target of Jan. 6 conspiracy theories, gets a year of probation for his Capitol riot role | AP News'],
      dtype=object)

## API (Tumblr)

Generally website owners do not like you scraping their sites. If done badly,
scarping can act like a DOS attack so you should be careful how often you make
calls to a site. Some sites want automated tools to access their data, so they
create [application programming interface
(APIs)](https://en.wikipedia.org/wiki/Application_programming_interface). An API
specifies a procedure for an application (or script) to access their data. Often
this is though a [representational state transfer
(REST)](https://en.wikipedia.org/wiki/Representational_state_transfer) web
service, which just means if you make correctly formatted HTTP requests they
will return nicely formatted data.

A nice example for us to study is [Tumblr](https://www.tumblr.com), they have a
[simple RESTful API](https://www.tumblr.com/docs/en/api/v1) that allows you to
read posts without any complicated html parsing.

We can get the first 20 posts from a blog by making an http GET request to
`'http://{blog}.tumblr.com/api/read/json'`, were `{blog}` is the name of the
target blog. Lets try and get the posts from [http://lolcats-lol-
cat.tumblr.com/](http://lolcats-lol-cat.tumblr.com/) (Note the blog says at the
top 'One hour one pic lolcats', but the canonical name that Tumblr uses is in
the URL 'lolcats-lol-cat').

In [103]:
tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'))

print(r.text[:1000])

var tumblr_api_read = {"tumblelog":{"title":"One hour one pic lolcats","description":"","name":"lolcats-lol-cat","timezone":"Europe\/Paris","cname":false,"feeds":[],"uuid":"t:nXFqyQsaizVnIxVAm-ttlA"},"posts-start":0,"posts-total":3926,"posts-type":false,"posts":[{"id":"679413944568430592","url":"https:\/\/lolcats-lol-cat.tumblr.com\/post\/679413944568430592","url-with-slug":"https:\/\/lolcats-lol-cat.tumblr.com\/post\/679413944568430592\/cat-cats-kitty-gato-saturday-meow-katze","type":"photo","date-gmt":"2022-03-22 09:00:29 GMT","date":"Tue, 22 Mar 2022 10:00:29","bookmarklet":0,"mobile":0,"feed-item":"","from-feed-id":0,"unix-timestamp":1647939629,"format":"html","reblog-key":"MDkw6o5C","slug":"cat-cats-kitty-gato-saturday-meow-katze","is-submission":false,"like-button":"<div class=\"like_button\" data-post-id=\"679413944568430592\" data-blog-name=\"lolcats-lol-cat\" id=\"like_button_679413944568430592\"><iframe id=\"like_iframe_679413944568430592\" src=\"https:\/\/assets.tumblr.com\/

This might not look very good on first inspection, but it has far fewer angle
braces than html, which makes it easier to parse. What we have is
[JSON](https://en.wikipedia.org/wiki/JSON) a 'human readable' text based data
transmission format based on javascript. Luckily, we can readily convert it to a
python `dict`.

In [104]:
#We need to load only the stuff between the curly braces
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
print(d.keys())
print(len(d['posts']))

dict_keys(['tumblelog', 'posts-start', 'posts-total', 'posts-type', 'posts'])
20


If we read the [API specification](https://www.tumblr.com/docs/en/api/v1), we
will see there are a lot of things we can get if we add things to our GET
request. First we can retrieve posts by their id number. Let's first get post
`146020177084`.

In [105]:
r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'), params = {'id' : 146020177084})
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
d['posts'][0].keys()
d['posts'][0]['photo-url-1280']

with open('lolcat.gif', 'wb') as f:
    gifRequest = requests.get(d['posts'][0]['photo-url-1280'], stream = True)
    f.write(gifRequest.content)

<img src='lolcat.gif'>

Such beauty; such vigor (If you can't see it you have to refresh the page). Now
we could retrieve the text from all posts as well
as related metadata, like the post date, caption or tags. We could also get
links to all the images.

In [106]:
#Putting a max in case the blog has millions of images
#The given max will be rounded up to the nearest multiple of 50
def tumblrImageScrape(blogName, maxImages = 200):
    #Restating this here so the function isn't dependent on any external variables
    tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

    #There are a bunch of possible locations for the photo url
    possiblePhotoSuffixes = [1280, 500, 400, 250, 100]

    #These are the pieces of information we will be gathering,
    #at the end we will convert this to a DataFrame.
    #There are a few other datums we could gather like the captions
    #you can read the Tumblr documentation to learn how to get them
    #https://www.tumblr.com/docs/en/api/v1
    postsData = {
        'id' : [],
        'photo-url' : [],
        'date' : [],
        'tags' : [],
        'photo-type' : []
    }

    #Tumblr limits us to a max of 50 posts per request
    for requestNum in range(maxImages // 50):
        requestParams = {
            'start' : requestNum * 50,
            'num' : 50,
            'type' : 'photo'
        }
        r = requests.get(tumblrAPItarget.format(blogName), params = requestParams)
        requestDict = json.loads(r.text[len('var tumblr_api_read = '):-2])
        for postDict in requestDict['posts']:
            #We are dealing with uncleaned data, we can't trust it.
            #Specifically, not all posts are guaranteed to have the fields we want
            try:
                postsData['id'].append(postDict['id'])
                postsData['date'].append(postDict['date'])
                postsData['tags'].append(postDict['tags'])
            except KeyError as e:
                raise KeyError("Post {} from {} is missing: {}".format(postDict['id'], blogName, e))

            foundSuffix = False
            for suffix in possiblePhotoSuffixes:
                try:
                    photoURL = postDict['photo-url-{}'.format(suffix)]
                    postsData['photo-url'].append(photoURL)
                    postsData['photo-type'].append(photoURL.split('.')[-1])
                    foundSuffix = True
                    break
                except KeyError:
                    pass
            if not foundSuffix:
                #Make sure your error messages are useful
                #You will be one of the users
                raise KeyError("Post {} from {} is missing a photo url".format(postDict['id'], blogName))

    return pandas.DataFrame(postsData)
tumblrImageScrape('lolcats-lol-cat', 50)

,id,photo-url,date,tags,photo-type
0,679413944568430592,https://64.media.tumblr.com/1fb7e346f39e428540...,"Tue, 22 Mar 2022 10:00:29","[gif, lolcat, lolcats, cat, funny, cats, kitty...",gif
1,662815854023655425,https://64.media.tumblr.com/021eac8fbcafbb00a5...,"Mon, 20 Sep 2021 06:00:56","[gif, lolcat, lolcats, cat, funny]",gif
2,662778109891952640,https://64.media.tumblr.com/8c0517adb8c71e4a3d...,"Sun, 19 Sep 2021 20:01:00","[cat, cats, lol, lolcat, lolcats]",png
3,662657302700146688,https://64.media.tumblr.com/061d27cda309d5c809...,"Sat, 18 Sep 2021 12:00:50","[cat, cats, lol, lolcat, lolcats]",jpg
4,662513901538246656,https://64.media.tumblr.com/80584a9d1ff4ddc4fc...,"Thu, 16 Sep 2021 22:01:32","[cat, cats, lol, lolcat, lolcats]",png
5,662257177983090688,https://64.media.tumblr.com/893b320cd2e8970a20...,"Tue, 14 Sep 2021 02:01:01","[cat, cats, lol, lolcat, lolcats]",jpg
6,662166591527698432,https://64.media.tumblr.com/c7f0a0a9184e480e15...,"Mon, 13 Sep 2021 02:01:11","[cat, cats, lol, lolcat, lolcats]",jpg
7,662113740899090432,https://64.media.tumblr.com/07f7be7f71917a6049...,"Sun, 12 Sep 2021 12:01:09","[cat, cats, lol, lolcat, lolcats]",jpg
8,661955166248026112,https://64.media.tumblr.com/205f030c48d31f8960...,"Fri, 10 Sep 2021 18:00:40","[cat, cats, lol, lolcat, lolcats]",jpg
9,661894830378614784,https://64.media.tumblr.com/c463bff883fec2045b...,"Fri, 10 Sep 2021 02:01:39","[cat, cats, lol, lolcat, lolcats]",png


Now we have the urls of a bunch of images and can run OCR on them to gather
compelling meme narratives, accompanied by cats.

# Files

What if the text we want isn't on a webpage? There are a many other sources of
text available, typically organized into *files*.

## Raw text (and encoding)

The most basic form of storing text is as a _raw text_ document. Source code
(`.py`, `.r`, etc) is usually raw text as are text files (`.txt`) and those with
many other extension (e.g., .csv, .dat, etc.). Opening an unknown file with a
text editor is often a great way of learning what the file is.

We can create a text file in python with the `open()` function

In [107]:
#example_text_file = 'sometextfile.txt'
#stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols \u2421 \u241B \u20A0 \u20A1 \u20A2 \u20A3 \u0D60\n'
stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ\n'

with open(example_text_file, mode = 'w', encoding='utf-8') as f:
    f.write(stringToWrite)

Notice the `encoding='utf-8'` argument, which specifies how we map the bits from
the file to the glyphs (and whitespace characters like tab (`'\t'`) or newline
(`'\n'`)) on the screen. When dealing only with latin letters, arabic numerals
and the other symbols on America keyboards you usually do not have to worry
about encodings as the ones used today are backwards compatible with
[ASCII](https://en.wikipedia.org/wiki/ASCII), which gives the binary
representation of 128 characters.

Some of you, however, will want to use other characters (e.g., Chinese
characters). To solve this there is
[Unicode](https://en.wikipedia.org/wiki/Unicode) which assigns numbers to
symbols, e.g., 041 is `'A'` and 03A3 is `'Σ'` (numbers starting with 0 are
hexadecimal). Often non/beyond-ASCII characters are called Unicode characters.
Unicode contains 1,114,112 characters, about 10\% of which have been assigned.
Unfortunately there are many ways used to map combinations of bits to Unicode
symbols. The ones you are likely to encounter are called by Python _utf-8_,
_utf-16_ and _latin-1_. _utf-8_ is the standard for Linux and Mac OS while both
_utf-16_ and _latin-1_ are used by windows. If you use the wrong encoding,
characters can appear wrong, sometimes change in number or Python could raise an
exception. Lets see what happens when we open the file we just created with
different encodings.

In [108]:
with open(example_text_file, encoding='utf-8') as f:
    print("This is with the correct encoding:")
    print(f.read())

with open(example_text_file, encoding='latin-1') as f:
    print("This is with the wrong encoding:")
    print(f.read())

This is with the correct encoding:
A line
Another line
A line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ

This is with the wrong encoding:
A line
Another line
A line with a few unusual symbols â¡ â â  â¡ â¢ â£ àµ 



Notice that with _latin-1_ the unicode characters are mixed up and there are too
many of them. You need to keep in mind encoding when obtaining text files.
Determining the encoding can sometime involve substantial work.

We can also load many text files at once. Lets start by looking at the Shakespeare files in the `data` directory

In [112]:
!mkdir ./data/Shakespeare
!wget -P ./data/Shakespeare https://github.com/UChicago-Computational-Content-Analysis/Homework-Notebooks-2024-Winter/blob/main/data/Shakespeare/midsummer_nights_dream.txt

mkdir: cannot create directory ‘./data/Shakespeare’: No such file or directory
--2024-01-11 23:30:31--  https://github.com/UChicago-Computational-Content-Analysis/Homework-Notebooks-2024-Winter/blob/main/data/Shakespeare/midsummer_nights_dream.txt
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 125150 (122K) [text/plain]
Saving to: ‘./data/Shakespeare/midsummer_nights_dream.txt’

midsummer_nights_dr 100%[===================>] 122.22K   662KB/s    in 0.2s    

2024-01-11 23:30:31 (662 KB/s) - ‘./data/Shakespeare/midsummer_nights_dream.txt’ saved [125150/125150]



In [113]:
with open('./data/Shakespeare/midsummer_nights_dream.txt') as f:
    midsummer = f.read()
print(midsummer[-700:])

nBanner":false,"actionsOnboardingTip":null},"truncated":false,"viewable":true,"workflowRedirectUrl":null,"symbols":{"timed_out":false,"not_analyzed":true,"symbols":[]}},"copilotInfo":null,"copilotAccessAllowed":false,"csrf_tokens":{"/UChicago-Computational-Content-Analysis/Homework-Notebooks-2024-Winter/branches":{"post":"_NATzHWBXLacpgs2W9jPN4gzOMguC7zGl6shd8DjTBTDvmZSokwKYPzls3cyDsQMs2p_K73Bp_67M4uwE8Rgpg"},"/repos/preferences":{"post":"FQa5Tql7r7UR5hPiydziRbNFnaSrwKieG7cc8e8ujGX2Q4CfJSbQiEcDEXj1JfIaQXYBUtu0fUy0_ZWV-Porwg"}}},"title":"Homework-Notebooks-2024-Winter/data/Shakespeare/midsummer_nights_dream.txt at main · UChicago-Computational-Content-Analysis/Homework-Notebooks-2024-Winter"}


By the way, depending on your working directory, you might get errors such as: [Errno 2] No such file or directory: '../data/Shakespeare/midsummer_nights_dream.txt.' Don't panic, it's nothing, just check your working directory.

Then to load all the files in `./data/Shakespeare` we can use a for loop with `scandir`:

In [114]:
targetDir = './data/Shakespeare' #Change this to your own directory of texts
shakespearText = []
shakespearFileName = []

for file in (file for file in os.scandir(targetDir) if file.is_file() and not file.name.startswith('.')):
    with open(file.path, encoding="utf-8") as f:
        shakespearText.append(f.read())
    shakespearFileName.append(file.name)

Then we can put them all in pandas DataFrame

In [115]:
shakespear_df = pandas.DataFrame({'text' : shakespearText}, index = shakespearFileName)
shakespear_df

,text
midsummer_nights_dream.txt,"{""payload"":{""allShortcutsEnabled"":false,""fileT..."


Getting your text in a format like this is the first step of most analysis

## PDF

Another common way text will be stored is in a PDF file. First we will download
a pdf in Python. To do that lets grab a chapter from
_Speech and Language Processing_, chapter 21 is on Information Extraction which
seems apt. It is stored as a pdf at [https://web.stanford.edu/~jurafsky/slp3/21.
pdf](https://web.stanford.edu/~jurafsky/slp3/21.pdf) although we are downloading
from a copy just in case Jurafsky changes their website.

In [116]:
#information_extraction_pdf = 'https://github.com/KnowledgeLab/content_analysis/raw/data/21.pdf'

infoExtractionRequest = requests.get(information_extraction_pdf, stream=True)
print(infoExtractionRequest.text[:1000])

%PDF-1.3
%���������
4 0 obj
<< /Length 5 0 R /Filter /FlateDecode >>
stream
x�]۶�F�}�W�c����T���C�i�<t�bM�f
Tn�<3_��CDf��J�N�i��#�%.;.���	?߄��7�]8������ux��}޾m����y��bǾ����!���$�Ǯ���C��F�����p���5��1��1�P<�{�$��/$�P�s�v��PgH?�����Q�~�*�:l��ˇ�m�ǰ��C�l����܊��E�����!�^�y��m�$�Ý���wۡل׼�6w���ī�K�~؞���r��~?�ˡkO�;6IH�9{ԡ��� ]?�E�E��~���.l������+��W�_�����C��S�|�~C��N�3ӛB`8�ޚj9���AZ��0�d�l^������SY	�Ƨ��>q�ۇ&
����.�����0����; ��>a8�$w�p��p����ST���. �7��@����)��&1�|���WՃ jOv�G2b�L8I��N�@gǍ������O�C��������IN@@�����}�8��+L����a�&ү�o�V(���0���+5�
SfS&��<�2�����>l�V��&��=4⇤=�W��<�JMo����"����d�C����[vY�|K{_ܔ\����%H�/@'�QA�+D�l��c��L�G�.��	�̎�V�:f>���AwK���o$`D��bE45�0�%th6h�����>*�2vQd�+M��Y}�Q���u�[���N�o'b��/u�.r'Z���J�e8�v��;��{T�	�����^8�  l<�E�<���b�����C8j��f��xB>K�����|w��f�|?�s̭��Y�'�Ip&�"�A���f�?�!IYi���U�"��y;���#��e3)�+B�&����<E9I�g�/]"D��yfC;e����Y^�z ��s'�)/�X�-HY��<ˬ�ݰ

It says `'pdf'`, so thats a good sign. The rest though looks like we are having
issues with an encoding. The random characters are not caused by our encoding
being wrong, however. They are cause by there not being an encoding for those
parts at all. PDFs are nominally binary files, meaning there are sections of
binary that are specific to pdf and nothing else so you need something that
knows about pdf to read them. To do that we will be using
[`PyPDF2`](https://github.com/mstamy2/PyPDF2), a PDF processing library for
Python 3.


Because PDFs are a very complicated file format pdfminer requires a large amount
of boilerplate code to extract text, we have written a function that takes in an
open PDF file and returns the text so you don't have to.

In [38]:
def readPDF(pdfFile):
    #Based on code from http://stackoverflow.com/a/20905381/4955164
    #Using utf-8, if there are a bunch of random symbols try changing this
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams, codec = codec)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    return returnedString

First we need to take the response object and convert it into a 'file like'
object so that pdfminer can read it. To do this we will use `io`'s `BytesIO`.

In [118]:
infoExtractionBytes = io.BytesIO(infoExtractionRequest.content)

Now we can give it to pdfminer.

In [119]:
print(readPDF(infoExtractionBytes)[:550])

Department of  Sociology 

THE UNIVERSITY OF CHICAGO 

SOCIOLOGY 40133 

Computational Content Analysis 

Friday 1:00 – 3:50pm 
Winter 2017-2018 
Classroom: Harper Memorial 130       
http://chalk.uchicago.edu/ 

 

                                                                                           

          Office: McGiffert 210 
                                                    Tel.: 834-3612; jevans@uchicago.edu 
                                  Office Hours: Thursday 12:30-2:30pm 

     

        James A. Evans            

    


From here we can either look at the full text or fiddle with our PDF reader and
get more information about individual blocks of text.

## Word Docs

The other type of document you are likely to encounter is the `.docx`, these are
actually a version of [XML](https://en.wikipedia.org/wiki/Office_Open_XML), just
like HTML, and like HTML we will use a specialized parser.

For this class we will use [`python-docx`](https://python-
docx.readthedocs.io/en/latest/) which provides a nice simple interface for
reading `.docx` files

In [32]:
#example_docx = 'https://github.com/KnowledgeLab/content_analysis/raw/data/example_doc.docx'

r = requests.get(example_docx, stream=True)
d = docx.Document(io.BytesIO(r.content))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

 
 

Accessing the Research Computing Center Resources

To connect to the midway compute cluster to access your home directory and the macs60000 storage space, and utilize the HPC resources, you will either use a terminal client (with or without X11 forwarding capabilities) or the Linux remote desktop server software client (Thinlinc) to connect to the midway cluster. To submit jobs, monitor jobs, browse directories or do other computing you will need to connect through either the terminal or remote desktop. Setup and utilization of these clients will be discussed below in the context of your local platform’s architecture.
SSH Client Setup & Remote Desktop Server


This procedure uses the `io.BytesIO` class again, since `docx.Document` expects
a file. Another way to do it is to save the document to a file and then read it
like any other file. If we do this we can either delete the file afterwords, or
save it and avoid downloading the following time.

This function is useful as a part of many different tasks so it and others like it will be added to the helper package `lucem_illud` so we can use it later without having to retype it.

In [33]:
def downloadIfNeeded(targetURL, outputFile, **openkwargs):
    if not os.path.isfile(outputFile):
        outputDir = os.path.dirname(outputFile)
        #This function is a more general os.mkdir()
        if len(outputDir) > 0:
            os.makedirs(outputDir, exist_ok = True)
        r = requests.get(targetURL, stream=True)
        #Using a closure like this is generally better than having to
        #remember to close the file. There are ways to make this function
        #work as a closure too
        with open(outputFile, 'wb') as f:
            f.write(r.content)
    return open(outputFile, **openkwargs)

This function will download, save and open `outputFile` as `outputFile` or just
open it if `outputFile` exists. By default `open()` will open the file as read
only text with the local encoding, which may cause issues if its not a text
file.

In [34]:
try:
    d = docx.Document(downloadIfNeeded(example_docx, example_docx_save))
except Exception as e:
    print(e)

File is not a zip file


We need to tell `open()` to read in binary mode (`'rb'`), this is why we added
`**openkwargs`, this allows us to pass any keyword arguments (kwargs) from
`downloadIfNeeded` to `open()`.

In [35]:
d = docx.Document(downloadIfNeeded(example_docx, example_docx_save, mode = 'rb'))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

 
 

Accessing the Research Computing Center Resources

To connect to the midway compute cluster to access your home directory and the macs60000 storage space, and utilize the HPC resources, you will either use a terminal client (with or without X11 forwarding capabilities) or the Linux remote desktop server software client (Thinlinc) to connect to the midway cluster. To submit jobs, monitor jobs, browse directories or do other computing you will need to connect through either the terminal or remote desktop. Setup and utilization of these clients will be discussed below in the context of your local platform’s architecture.
SSH Client Setup & Remote Desktop Server


Now we can read the file with `docx.Document` and not have to wait for it to be
downloaded every time.

# <font color="93FFE8">Exercise 3</font>
<font color="93FFE8">Construct cells immediately below this that extract and organize textual content from text, PDF or Word into a pandas dataframe.</font>


In this part, I will download a pdf for the novel "The Great Gastby" by F. Scott Fitzgerald, and extracts all lines contain the word "Gastby".

In [36]:
import requests

# request the pdf
url = "https://pdcrodas.webs.ull.es/naturalismo/FitzgeraldTheGreatGastby.pdf"
response = requests.get(url, stream=True)

with open("FitzgeraldTheGreatGastby.pdf", "wb") as f:
    f.write(response.content)

In [39]:
with open("FitzgeraldTheGreatGastby.pdf", "rb") as f:
    text = readPDF(f)

# process text
Daisy_lines = []
for l in text.split("."):
    if "Daisy" in l:
        Daisy_lines.append(re.sub(r'\n', '', l))

In [40]:
df_Daisy = pandas.DataFrame(Daisy_lines, columns = ["text"])
df_Daisy

,text
0,Daisy was my second cousin once remove...
1,"This was a permanent move, said Daisy over th..."
2,"The other girl, Daisy, made an attempt to ris..."
3,(I’ve heard it said that Daisy’s murmu...
4,"” “Oh, I’ll stay in the East, don’t you..."
...,...
173,I see now that this has been a story of the W...
174,He threw dust into your eyes just like he did...
175,"They were careless people, Tom and Dais..."
176,One of the taxi drivers in the village never ...


# <font color="93FFE8">Exercise 4</font>

<font color="93FFE8">In the two cells immediately following, describe a possible project (e.g., it might end up being your final project, but need not be if you are still searching): **WHAT** you will analyze--the texts you will select and the social game, world and actors you intend to learn about through your analysis (<100 words); **WHY** you will analyze these texts to learn about that context--justify the rationale behind your proposed sample design for this project, based on the readings. What is the social game, social work, or social actors about whom you are seeking to make inferences? What are the virtues of your proposed sample with respect to your research questions? What are its limitations? What are alternatives? What would be a reasonable path to "scale up" your sample for further analysis (i.e., high-profile publication)? (<150 words)? [**Note**: your individual or collective project will change over the course of the quarter as new data and/or analysis opportunities arise or if old ones fade away.]

## ***What?***
I propose to develop a type of spider that crawl the internet for related content sharing the similar worldview or opinion. Essentially, this will be a recursive crawler that only crawl the content that is relevant or likely deemed as trustworthy by the initial source (start point). The main sources will be the web content, whether articles or user pages.

## ***Why?***
Rationale:
The rationale behind this project is that similar content or user tend to connect together, which make the crawling feasible and efficient. I also propose to leverage the LLM’s ability to process unstructured text on the webpage, as individual webpages may be highly unique. This is an initial exploratory project, which does not necessarily answer any specific questions: instead, this project aims to collect data points related to a specific group or opinion for future analysis.

Limitation:
I believe that the most difficult part is to model the trust or worldview of the users or content. If the model is performing badly, after a couple jumps, the content may hold very different view than the initial source. In the first lecture, professor discussed a couple papers/works about modeling worldview or predict future opinion of a group, which I plan to read and to see if anything I could leverage.



### Sources:

Other popular sources for internet data:

[reddit](https://www.reddit.com/) - https://praw.readthedocs.io/en/v2.1.21/

[twitter](https://twitter.com/) - https://pypi.org/project/python-twitter/

[project gutenburg](https://www.gutenberg.org/) - https://github.com/ageitgey/Gutenberg

